In [7]:
SOS_token = 0
EOS_token = 1

In [2]:
import json
import random

path = "data/msmarco_2wellformed/dev_v2.0_well_formed.json"

def loadDataset(path,limit=100000000000):
    print("Loading dataset...")
    dataset = list()
    i = 0
    for line in open(path, 'r'):
        all_data = json.loads(line)
        for data in all_data:
            if i <limit:
                dataset.append(data)
                i += 1
            else:
                break
    print("Loaded %d pieces of data"%len(dataset))
    return dataset

dataset = loadDataset(path)
for i in range(20):
    print('')
    x = random.choice(dataset)
    print(x['query'])
    print(x['wellFormedAnswers'][0])

Loading dataset...
Loaded 14261 pieces of data

what chemical in capacitor leak
Electrolyte chemical is in capacitor leak.

what county is saucier
Saucier is in Harrison County.

what is food?
Food is any substance consumed to provide nutritional support for an organism.  

what phylicia rashad says about the cosby trial
Phylicia Rashad is speaking out about the Bill Cosby sexual assault allegations, defending her longtime friend. “I love him,” she told Roger Friedman's Showbiz 411 earlier this week at Paramount's lunch for the movie Selma.

what is an example of a certified public record
The example of a certified public record is Marriage Records.

are breitling watches made in japan
Yes, Breitling watches are not made in Japan.

population socorro new mexico
The population of Socorro, New Mexico is 8,906.

what hazchem indicates
The top-left section of the plate gives the Emergency Action Code telling the fire brigade what actions to take if there's an accident is indicated in Hazch

In [21]:
import re

# def processFinalPeriod(sentence1):
# 	if (sentence1[len(sentence1)-1] == "."):
# 		sentence1 = sentence1[:len(sentence1)-2] + " ."

def insertCharIfSeq(sentence1,c,seq):
    i = 0
    indexes = [m.start() for m in re.finditer(seq, sentence1)]
    for index in indexes:
        sentence1 = sentence1[:index+i] + c + sentence1[index+i:]
        i += 1
    return sentence1

def processContractions(sentence1):
    sentence1 = insertCharIfSeq(sentence1," ","'s")
    sentence1 = insertCharIfSeq(sentence1," ","'m")
    sentence1 = insertCharIfSeq(sentence1," ","'ll")
    sentence1 = insertCharIfSeq(sentence1," ","'ve")
    sentence1 = insertCharIfSeq(sentence1," ","'re")
    sentence1 = insertCharIfSeq(sentence1," ","'d")
    return sentence1


def processNegatives(sentence1):
    i = 0
    indexes = [m.start() for m in re.finditer("can't", sentence1)]
    for index in indexes:
        sentence1 = sentence1[:index+i+3] + sentence1[index+i+2:]
        i += 1
    return insertCharIfSeq(sentence1," ","n't")

## WHAT TO DO WITH HYPHENS ??

def tokenizeSentence(sentence1):
#     processFinalPeriod(sentence1)
    sentence1 = processContractions(sentence1)
    sentence1 = processNegatives(sentence1)
    s = sentence1.lower()
    s = re.sub('''([.,!"?$;:/#`()])''', r' \1 ', s)
    s = re.sub('\s{2,}', ' ', s)
    s = s.split()
#     s = processHyphenIfUnknownWords(s,glove)
    s.append('</s>')
    s = ['<s>'] + s
    return s

def testTokenizeSentence():
    sentence1 = "I can't open the door because the 30-year old/blond-hair guy doesn't want to let me in. He's mean, isn't he? I can't go in! There's no other way!"
    words = tokenizeSentence(sentence1)
    print(words)
    
testTokenizeSentence()

['<s>', 'i', 'can', "n't", 'open', 'the', 'door', 'because', 'the', '30-year', 'old', '/', 'blond-hair', 'guy', 'does', "n't", 'want', 'to', 'let', 'me', 'in', '.', 'he', "'s", 'mean', ',', 'is', "n't", 'he', '?', 'i', 'can', "n't", 'go', 'in', '!', 'there', "'s", 'no', 'other', 'way', '!', '</s>']


In [38]:
import math

def tokenizeDataset(dataset):
    tokenized_dataset = list()
    len_dataset = len(dataset)
    print_every = math.floor(len_dataset / 10)
    print('Tokenizing dataset...')
    for i, data in enumerate(dataset):
        tokenized_data = dict()
        tokenized_data['answers'] = list()
        tokenized_data['wellFormedAnswers'] = list()
        tokenized_data['passages'] = list()
        for answer in data['answers']:
            tokenized_data['answers'].append(tokenizeSentence(answer))
        for wf_answer in data['wellFormedAnswers']:
            tokenized_data['wellFormedAnswers'].append(tokenizeSentence(wf_answer))
        for passage in data['passages']:
            passage_dict = dict()
            passage_dict['passage_text'] = tokenizeSentence(passage['passage_text'])
            tokenized_data['passages'].append(passage_dict)
        if i>0 and i%print_every == 0:
            print('... %d%%'%(i/print_every*10),end="\r")

tokenizeDataset(dataset)

Tokenizing dataset...


In [5]:
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 256
VOC_SIZE = 50000 #for both source and target
OUTPUT_SIZE = VOC_SIZE #10 # ?? 
BASELINE_VOC_SIZE = 150000
MAX_LENGTH = 400

In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable


class EncoderDecoderLSTM(nn.Module):
    
    def __init__(self,embedding_size=EMBEDDING_SIZE,hidden_size=HIDDEN_SIZE,voc_size=VOC_SIZE):
        super(EncoderLSTM,self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.voc_size = voc_size
        
        # Encoder
        self.embedding = nn.Embedding(voc_size, embedding_size)
        self.encoder_biltsm = nn.LSTM(embedding_size, hidden_size, num_layers =1, bidirectional=True)
        
        # Decoder
        self.decoder_bilstm = nn.LSTM(self.embedding_size,self.hidden_size, num_layers=1, bidirectional = True)
        self.attn_W = nn.Linear(hidden_size *2, max_length) #?
        self.attn_v = nn.Linear(hidden_size*2, hidden_size)
        self.lin_V1 = nn.Linear(hidden_size*2, hidden_size)
        self.lin_V2 = nn.Lineal(hidden_size, output_size)
        
    def init_hidden(self):
        return Variable(torch.zeros(1,1,self.hidden_size))
    
    def encoder_forward(self, input, hidden):
        emb = self.embedding(input).view(1,1,-1)
        output, hidden = self.encoder_bilstm(input, hidden)
        return output, hidden
    
    def decoder_forward(self,input,hidden):
        emb = self.embedding(token).view(1,1,-1)
        output, hidden = self.decoder_bilstm(emb,hidden)
        return output, hidden
    
    
    def forward(self,pair):        
        text = pair[0]
        summary = pair[1]
        
        text_length = len(text)
        encoder_hidden = self.initHidden()        
        h = Variable(torch.zeros(self.text_length, self.hidden_size))        
        for ei in range(text_length):
            encoder_output, encoder_hidden = encoder_forward(text[ei],encoder_hidden)
            h[ei] = encoder_output
                    
        summary_length = len(summary)
        decoder_hidden = self.initHidden()        
        s = Variable(torch.zeros(self.summary_length, self.hidden_size))        
        decoder_input = Varible(torch.longTensor([[SOS_token]]))        
        for di in range(summary_length):
            decoder_output, decoder_hidden = decoder_forward(decoder_input, decoder_hidden)
            s[di] = decoder_hidden
            decoder_input = summary[di]
            
        #attention distribution
        Wh_Ws_d = self.attn_W(torch.cat((h,s),1))
        e = self.attn_v(Wh_Ws_d)
        a = F.softmax(e, dim=1)
        h_star = torch.bmm(a,h)
        
        #vocabulary distribution
        v1 = self.lin_V1(torch.cat((s,h_star),dim=1))
        v2 = self.lin_V2(v1)
        Pvocab = F.softmax(v2,dim=1)
        
        # find target word
        